## Démarrage de la session Spark 

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683198541216_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683198541216_0001,pyspark,idle,Link,Link,None,✔


## Installation des packages

## Import des librairies

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Définition des PATH pour charger les images et enregistrer les résultats

In [5]:
PATH = 's3://projet-8-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://projet-8-data
PATH_Data:   s3://projet-8-data/Test
PATH_Result: s3://projet-8-data/Results

## Chargement des données

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://projet-8-dat...|2023-04-29 12:37:24|  5941|[FF D8 FF E0 00 1...|
|s3://projet-8-dat...|2023-04-29 12:37:24|  5890|[FF D8 FF E0 00 1...|
|s3://projet-8-dat...|2023-04-29 12:37:25|  5863|[FF D8 FF E0 00 1...|
|s3://projet-8-dat...|2023-04-29 12:37:24|  5861|[FF D8 FF E0 00 1...|
|s3://projet-8-dat...|2023-04-29 12:37:23|  5843|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------+--------------+
|path                                             |label         |
+-------------------------------------------------+--------------+
|s3://projet-8-data/Test/Apple Golden 1/72_100.jpg|Apple Golden 1|
|s3://projet-8-data/Test/Apple Golden 1/74_100.jpg|Apple Golden 1|
|s3://projet-8-data/Test/Apple Golden 1/75_100.jpg|Apple Golden 1|
|s3://projet-8-data/Test/Apple Golden 1/73_100.jpg|Apple Golden 1|
|s3://projet-8-data/Test/Apple Golden 1/63_100.jpg|Apple Golden 1|
+-------------------------------------------------+--------------+
only showing top 5 rows

None

## Préparation du modèle

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 0s 0us/step

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1683198541216_0001/container_1683198541216_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

## Exécutions des actions d'extractions de features

In [15]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://projet-8-data/Results

In [18]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargement des données enregistrées et validation du résultat

In [24]:
df = spark.read.parquet(PATH_Result)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+
|                path|             label|            features|
+--------------------+------------------+--------------------+
|s3://projet-8-dat...|          Beetroot|[1.3021957, 0.057...|
|s3://projet-8-dat...|      Cactus fruit|[0.42030877, 0.05...|
|s3://projet-8-dat...|Apple Red Yellow 1|[0.28037623, 0.00...|
|s3://projet-8-dat...|    Apple Braeburn|[0.6281863, 0.039...|
|s3://projet-8-dat...|    Apple Golden 3|[0.39719278, 0.01...|
|s3://projet-8-dat...|    Apple Braeburn|[0.86953306, 0.00...|
|s3://projet-8-dat...|    Apple Golden 3|[0.40969056, 0.10...|
|s3://projet-8-dat...|       Apple Red 2|[0.848838, 0.0979...|
|s3://projet-8-dat...|      Avocado ripe|[2.181823, 0.0911...|
|s3://projet-8-dat...|        Banana Red|[0.23249891, 0.72...|
|s3://projet-8-dat...|         Blueberry|[0.67983824, 0.16...|
|s3://projet-8-dat...|            Banana|[1.4596508, 0.0, ...|
|s3://projet-8-dat...|       Apple Red 2|[0.9870568, 0.

In [29]:
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.functions import udf

ud_f = udf(lambda r: Vectors.dense(r), VectorUDT())
df = df.withColumn('features', ud_f('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
from pyspark.ml.feature import StandardScaler, PCA


scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df)


df_scaled = scaler.transform(df)
df_scaled.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+--------------------+
|                path|             label|            features|      scaledFeatures|
+--------------------+------------------+--------------------+--------------------+
|s3://projet-8-dat...|          Beetroot|[1.30219566822052...|[1.10364723512824...|
|s3://projet-8-dat...|      Cactus fruit|[0.42030876874923...|[-0.3722668309885...|
|s3://projet-8-dat...|Apple Red Yellow 1|[0.28037622570991...|[-0.6064560549163...|
|s3://projet-8-dat...|    Apple Braeburn|[0.62818628549575...|[-0.0243658119581...|
|s3://projet-8-dat...|    Apple Golden 3|[0.39719277620315...|[-0.4109534455323...|
+--------------------+------------------+--------------------+--------------------+
only showing top 5 rows

In [32]:
n_components = 40
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(df_scaled)

df_pca = pca.transform(df_scaled)
print('Explained Variance Ratio', sum(pca.explainedVariance.toArray()))
df_pca.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio 0.9567626430089331
+--------------------+------------------+--------------------+--------------------+--------------------+
|                path|             label|            features|      scaledFeatures|         pcaFeatures|
+--------------------+------------------+--------------------+--------------------+--------------------+
|s3://projet-8-dat...|          Beetroot|[1.30219566822052...|[1.10364723512824...|[-39.716085023405...|
|s3://projet-8-dat...|      Cactus fruit|[0.42030876874923...|[-0.3722668309885...|[-11.109107753582...|
|s3://projet-8-dat...|Apple Red Yellow 1|[0.28037622570991...|[-0.6064560549163...|[12.9292747556642...|
|s3://projet-8-dat...|    Apple Braeburn|[0.62818628549575...|[-0.0243658119581...|[12.4775292417174...|
|s3://projet-8-dat...|    Apple Golden 3|[0.39719277620315...|[-0.4109534455323...|[8.25967669410774...|
+--------------------+------------------+--------------------+--------------------+--------------------+
only showin

In [36]:
df_pca.write.mode("overwrite").parquet("s3://projet-8-data/Results_pca")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…